In [2]:
import time
import json
import paho.mqtt.client as mqtt
import threading

In [4]:

# --- Configuration ---
MQTT_BROKER = "localhost"
MQTT_PORT = 1883
TOPIC = "test/qos/#"   # Matches test/qos/0, test/qos/1, test/qos/2
CLIENT_ID = "QoS_Test_Subscriber"

# Disconnect test parameters
DISCONNECT_AFTER = 20   # Disconnect after 20 seconds
DISCONNECT_DURATION = 15  # Stay disconnected for 15 seconds
TOTAL_TEST_TIME = 60    # Total test duration in seconds

# --- Global Counters ---
message_counts = {"test/qos/0": 0, "test/qos/1": 0, "test/qos/2": 0}
received_ids = {"test/qos/0": set(), "test/qos/1": set(), "test/qos/2": set()}
duplicate_counts = {"test/qos/0": 0, "test/qos/1": 0, "test/qos/2": 0}
total_messages = 0
is_connected = False

# --- MQTT Callbacks (API v2 signatures) ---
def on_connect(client, userdata, flags, reason_code, properties):
    global is_connected
    if reason_code == 0:
        print(f"Connected to MQTT broker at {time.strftime('%H:%M:%S')}")
        is_connected = True
    else:
        print(f"Connection failed: {reason_code}")
        is_connected = False
    client.subscribe([(TOPIC, 2)])
    print(f"  Subscribed to {TOPIC} with QoS 2")

def on_disconnect(client, userdata, flags, reason_code, properties=None):
    global is_connected
    is_connected = False
    print(f"Disconnected from broker at {time.strftime('%H:%M:%S')} (reason: {reason_code})")

def on_message(client, userdata, msg):
    global total_messages
    topic = msg.topic
    qos = msg.qos
    total_messages += 1

    # If a new subtopic appears, track it
    if topic not in message_counts:
        message_counts[topic] = 0
        received_ids[topic] = set()
        duplicate_counts[topic] = 0

    message_counts[topic] += 1

    # Payload parsing and duplicate detection
    try:
        payload = json.loads(msg.payload.decode("utf-8"))
        message_id = payload.get("message_id")
        if message_id is not None:
            if message_id in received_ids[topic]:
                duplicate_counts[topic] += 1
                print(f"Duplicate: {topic} | ID={message_id} | QoS={qos}")
            else:
                received_ids[topic].add(message_id)
    except json.JSONDecodeError:
        print(f"Non-JSON payload on {topic}")

    # Only print every 5th message to reduce clutter
    if message_counts[topic] % 5 == 0 or topic == "test/qos/0":
        print(f"  [{topic} | QoS {qos}] Count: {message_counts[topic]}")

# --- Main Execution ---
print("=" * 70)
print("  MQTT QoS Test with Simulated Disconnect")
print("=" * 70)
print(f"Test plan:")
print(f"  • Run for {DISCONNECT_AFTER}s connected")
print(f"  • Disconnect for {DISCONNECT_DURATION}s (messages published but not received)")
print(f"  • Reconnect and run for remaining time")
print(f"  • Total test time: {TOTAL_TEST_TIME}s")
print("=" * 70)

client = mqtt.Client(
    client_id=CLIENT_ID,
    clean_session=False,  # CRITICAL: Persistent session for QoS 1 & 2
    protocol=mqtt.MQTTv311,
    callback_api_version=mqtt.CallbackAPIVersion.VERSION2
)

client.on_connect = on_connect
client.on_disconnect = on_disconnect
client.on_message = on_message

try:
    # Initial connection with clean start to clear old data
    print("\n[Phase 0] Clearing old session data...")
    temp_client = mqtt.Client(
        client_id=CLIENT_ID,
        clean_session=True,  # Clear any old queued messages
        protocol=mqtt.MQTTv311,
        callback_api_version=mqtt.CallbackAPIVersion.VERSION2
    )
    temp_client.connect(MQTT_BROKER, MQTT_PORT, 60)
    temp_client.disconnect()
    time.sleep(1)

    # Initial connection
    print("\n[Phase 1] Connecting to broker...")
    client.connect(MQTT_BROKER, MQTT_PORT, 60)
    client.loop_start()

    # Wait for connection and verify messages are coming
    time.sleep(3)
    if total_messages == 0:
        print("\n WARNING: No messages received after 3 seconds!")
        print("   Please verify Node-RED flow is deployed and running.")
        print("   Press Ctrl+C to exit, or wait to continue test anyway...")
        time.sleep(2)

    # Phase 1: Connected period
    print(f"\n[Phase 1] Connected - receiving messages for {DISCONNECT_AFTER}s...")
    time.sleep(DISCONNECT_AFTER - 5)  # Subtract the verification time

    # Phase 2: Disconnect
    print(f"\n[Phase 2] DISCONNECTING for {DISCONNECT_DURATION}s...")
    print(f"           Messages published during this time:")
    print(f"           • QoS 0: Will be LOST")
    print(f"           • QoS 1 & 2: Will be QUEUED by broker")
    client.loop_stop()
    client.disconnect()

    time.sleep(DISCONNECT_DURATION)

    # Phase 3: Reconnect
    remaining_time = TOTAL_TEST_TIME - DISCONNECT_AFTER - DISCONNECT_DURATION
    print(f"\n[Phase 3] RECONNECTING...")
    print(f"           Expecting queued QoS 1 & 2 messages to be delivered now")
    client.connect(MQTT_BROKER, MQTT_PORT, 60)
    client.loop_start()

    time.sleep(remaining_time)

except KeyboardInterrupt:
    print("\n\n Test interrupted by user")
except Exception as e:
    print(f"\n Connection error: {e}")
finally:
    try:
        client.loop_stop()
    except:
        pass
    try:
        if is_connected:
            client.disconnect()
    except:
        pass

# --- Print Final Results ---
print("\n" + "=" * 70)
print("             QoS Test Final Results")
print("=" * 70)
print(f"Total messages received: {total_messages}")
print(f"Disconnect duration: {DISCONNECT_DURATION}s (~{DISCONNECT_DURATION} messages lost for QoS 0)")
print()

# Use QoS 2 count as the target (baseline)
TARGET_SENT = message_counts.get("test/qos/2", 0)

print("{:<20} {:<10} {:<10} {:<12} {:<10}".format(
    "Topic", "Target", "Received", "Duplicates", "Loss %"
))
print("-" * 70)

for topic in sorted(message_counts.keys()):
    received = message_counts[topic]
    duplicates = duplicate_counts.get(topic, 0)

    # Calculate loss percentage
    if TARGET_SENT > 0:
        loss_pct = ((TARGET_SENT - received) / TARGET_SENT) * 100
    else:
        loss_pct = 0.0

    print(
        "{:<20} {:<10} {:<10} {:<12} {:<10.1f}".format(
            topic,
            TARGET_SENT if TARGET_SENT > 0 else "N/A",
            received,
            duplicates,
            loss_pct
        )
    )

print("\n" + "=" * 70)
print("Target is based on QoS 2 count (exactly-once delivery)")
print("=" * 70)
print("\n Expected Results:")
print(f"  • QoS 0 (test/qos/0): Should lose ~{DISCONNECT_DURATION} messages")
print(f"  • QoS 1 (test/qos/1): Should match QoS 2, may have duplicates")
print(f"  • QoS 2 (test/qos/2): Should receive all {TARGET_SENT} messages, no duplicates")

if TARGET_SENT == 0:
    print("\n WARNING: No messages received! Check:")
    print("  1. Node-RED flow is deployed and running")
    print("  2. Mosquitto broker is running")
    print("  3. Topics match (test/qos/0, test/qos/1, test/qos/2)")
    print("  4. No firewall blocking localhost:1883")

print()

  MQTT QoS Test with Simulated Disconnect
Test plan:
  • Run for 20s connected
  • Disconnect for 15s (messages published but not received)
  • Reconnect and run for remaining time
  • Total test time: 60s

[Phase 0] Clearing old session data...

 Connection error: [Errno 111] Connection refused

             QoS Test Final Results
Total messages received: 0
Disconnect duration: 15s (~15 messages lost for QoS 0)

Topic                Target     Received   Duplicates   Loss %    
----------------------------------------------------------------------
test/qos/0           N/A        0          0            0.0       
test/qos/1           N/A        0          0            0.0       
test/qos/2           N/A        0          0            0.0       

Target is based on QoS 2 count (exactly-once delivery)

 Expected Results:
  • QoS 0 (test/qos/0): Should lose ~15 messages
  • QoS 1 (test/qos/1): Should match QoS 2, may have duplicates
  • QoS 2 (test/qos/2): Should receive all 0 messages, 